In [1]:
# load libraries
import pandas as pd
import plotly.express as px
import plotly.offline as pyo
import re
import numpy as np

In [2]:
# load data
directory = "C:\\Users\\joshg\\OneDrive\\Desktop\\code_pudding\\code_pudding_v2\\code_pudding2"
df = pd.read_csv(directory + "\speed_dating_data.csv", encoding='latin1')

In [3]:
# viewing a sample of the data
print(df.info())
df.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Columns: 195 entries, iid to amb5_3
dtypes: float64(174), int64(13), object(8)
memory usage: 12.5+ MB
None


,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
177,18,8.0,1,16,1,1,10,10,NaN,5,...,6.0,7.0,8.0,6.0,9.0,NaN,NaN,NaN,NaN,NaN
1799,127,6.0,1,12,1,5,10,1,NaN,4,...,8.0,8.0,8.0,8.0,8.0,NaN,NaN,NaN,NaN,NaN
3275,227,14.0,1,28,2,9,20,9,4.0,6,...,7.0,9.0,9.0,8.0,9.0,NaN,NaN,NaN,NaN,NaN
349,30,10.0,0,19,2,2,16,15,NaN,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2600,193,10.0,1,20,1,8,10,7,5.0,3,...,7.0,7.0,12.0,6.0,8.0,NaN,NaN,NaN,NaN,NaN
1734,120,11.0,0,19,1,5,10,3,NaN,1,...,7.0,8.0,9.0,7.0,9.0,NaN,NaN,NaN,NaN,NaN
2471,180,7.0,0,13,1,8,10,2,2.0,2,...,9.0,7.0,9.0,9.0,9.0,NaN,NaN,NaN,NaN,NaN
5655,374,14.0,1,28,2,14,20,12,12.0,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7293,493,13.0,1,25,2,19,15,2,11.0,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4426,292,20.0,1,40,2,11,21,9,9.0,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# get rid of a bad entry
df = df.dropna(subset='pid')

## Feature Engineering

In [5]:
# number of dates per attendee
dates_count = df.groupby('iid')['iid'].count()

def get_dates_count(obs):
    return dates_count[obs['iid']]

df['dates_count'] = df.apply(get_dates_count, axis=1)

In [6]:
# check to see if above matches 'round'
df['sanity'] = df['dates_count'] - df['round']
df[df['sanity'] != 0][['iid', 'wave', 'pid', 'round', 'dates_count', 'sanity']]

,iid,wave,pid,round,dates_count,sanity
1746,122,5,112.0,10,9,-1
1747,122,5,113.0,10,9,-1
1748,122,5,114.0,10,9,-1
1749,122,5,115.0,10,9,-1
1750,122,5,116.0,10,9,-1
...,...,...,...,...,...,...
6799,453,17,435.0,11,10,-1
6800,453,17,436.0,11,10,-1
6801,453,17,437.0,11,10,-1
6802,453,17,438.0,11,10,-1


In [7]:
matches_given = df.groupby('iid')['dec'].sum()

def get_matches_given(obs):
    return matches_given[obs['iid']]

df['matches_given'] = df.apply(get_matches_given, axis=1)

In [8]:
matches = {}

for iid in df['iid'].unique():
    matches[iid] = []

def get_matches(obs):
    if obs['dec'] == 1:
        matches[obs['iid']].append(obs['pid'])
        print(obs['iid'],matches[obs['iid']], matches[obs['pid']])

df.apply(get_matches, axis=1)

matches

1 [11.0] []
1 [11.0, 12.0] []
1 [11.0, 12.0, 13.0] []
1 [11.0, 12.0, 13.0, 14.0] []
1 [11.0, 12.0, 13.0, 14.0, 15.0] []
1 [11.0, 12.0, 13.0, 14.0, 15.0, 17.0] []
1 [11.0, 12.0, 13.0, 14.0, 15.0, 17.0, 19.0] []
1 [11.0, 12.0, 13.0, 14.0, 15.0, 17.0, 19.0, 20.0] []
2 [14.0] []
2 [14.0, 17.0] []
2 [14.0, 17.0, 19.0] []
2 [14.0, 17.0, 19.0, 20.0] []
4 [14.0] []
4 [14.0, 19.0] []
4 [14.0, 19.0, 20.0] []
5 [14.0] []
5 [14.0, 16.0] []
5 [14.0, 16.0, 17.0] []
5 [14.0, 16.0, 17.0, 18.0] []
5 [14.0, 16.0, 17.0, 18.0, 19.0] []
5 [14.0, 16.0, 17.0, 18.0, 19.0, 20.0] []
6 [11.0] []
6 [11.0, 12.0] []
6 [11.0, 12.0, 14.0] []
6 [11.0, 12.0, 14.0, 17.0] []
6 [11.0, 12.0, 14.0, 17.0, 19.0] []
6 [11.0, 12.0, 14.0, 17.0, 19.0, 20.0] []
7 [14.0] []
7 [14.0, 17.0] []
7 [14.0, 17.0, 19.0] []
7 [14.0, 17.0, 19.0, 20.0] []
8 [11.0] []
8 [11.0, 12.0] []
8 [11.0, 12.0, 13.0] []
8 [11.0, 12.0, 13.0, 14.0] []
8 [11.0, 12.0, 13.0, 14.0, 15.0] []
8 [11.0, 12.0, 13.0, 14.0, 15.0, 16.0] []
8 [11.0, 12.0, 13.0, 14.0, 1

{1: [11.0, 12.0, 13.0, 14.0, 15.0, 17.0, 19.0, 20.0],
 2: [14.0, 17.0, 19.0, 20.0],
 3: [],
 4: [14.0, 19.0, 20.0],
 5: [14.0, 16.0, 17.0, 18.0, 19.0, 20.0],
 6: [11.0, 12.0, 14.0, 17.0, 19.0, 20.0],
 7: [14.0, 17.0, 19.0, 20.0],
 8: [11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0],
 9: [11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0],
 10: [13.0, 17.0, 19.0, 20.0],
 11: [],
 12: [7.0, 8.0, 9.0],
 13: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0],
 14: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0],
 15: [1.0, 2.0, 3.0, 4.0, 6.0, 7.0, 8.0, 9.0, 10.0],
 16: [1.0, 2.0, 3.0, 4.0, 6.0, 7.0, 8.0, 9.0],
 17: [4.0, 9.0],
 18: [2.0, 7.0, 8.0],
 19: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0],
 20: [8.0],
 21: [42.0, 49.0],
 22: [44.0, 45.0, 46.0, 48.0, 50.0, 53.0, 55.0],
 23: [40.0, 42.0, 43.0, 44.0, 45.0, 46.0, 49.0, 50.0],
 24: [46.0, 53.0],
 25: [],
 26: [],
 27: [46.0, 50.0],
 28: [46.0, 51.0, 53.0],
 29: [44.0, 45.0, 46.0, 48.0, 50.0, 51.0, 52.0,

In [9]:
def get_dec_o2(obs):
    if obs['iid'] in matches[obs['pid']]:
        return 1
    else:
        return 0

df['dec_o2'] = df.apply(get_dec_o2, axis=1)

In [10]:
matches_received = df.groupby('iid')['dec_o2'].sum()

def get_matches_received(obs):
    return matches_received[obs['iid']]

df['matches_received'] = df.apply(get_matches_received, axis=1)

In [11]:
#superfluous
'''def get_mutual_match(obs):
    return obs['match'] and obs['dec_o2']

df['mutual_match'] = df.apply(get_mutual_match, axis=1)'''

"def get_mutual_match(obs):\n    return obs['match'] and obs['dec_o2']\n\ndf['mutual_match'] = df.apply(get_mutual_match, axis=1)"

In [12]:
mutual_count = df.groupby('iid')['match'].sum()

def get_mutual_count(obs):
    return mutual_count[obs['iid']]
    
df['mutual_count'] = df.apply(get_mutual_count, axis=1)

In [13]:
df['match_given_rate'] = df['matches_given'] / df['dates_count']
df['match_receive_rate'] = df['matches_received'] / df['dates_count']
df['mutual_rate'] = df['mutual_count'] / df['matches_given']

In [14]:
def decode_genders(obs):
    if obs['gender'] == 1:
        return 'Male'
    else:
        return 'Female'
    
df['gender_name'] = df.apply(decode_genders, axis=1)

In [15]:
goals = {1.0: 'Seemed like a fun night out',
             2.0: 'To meet new people',
             3.0: 'To get a date',
             4.0: 'Looking for a serious relationship',
             5.0: 'To say I did it',
             6.0: 'Other'}

def decode_goal(obs):
    if obs['goal'] in list(goals.keys()):
        return goals[obs['goal']]
    
df['goal_dc'] = df.apply(decode_goal, axis=1)

In [16]:
career_codes = {1.0: 'Lawyer',
                2.0: 'Academic/Research',
                3.0: 'Psychologist',
                4.0: 'Doctor/Medicine',
                5.0: 'Engineer',
                6.0: 'Creative Arts/Entertainment',
                7.0: 'Banking/Consulting/Finance/Marketing/Business/CEO/Entrepreneur/Admin',
                8.0: 'Real Estate',
                9.0: 'International/Humanitarian Affairs',
                10.0: 'Undecided',
                11.0: 'Social Work',
                12.0: 'Speech Pathology',
                13.0: 'Politics',
                14.0: 'Pro sports/Athletics',
                15.0: 'Other',
                16.0: 'Journalism',
                17.0: 'Architecture'}

career_codes = pd.Series(career_codes)

def decode_careers(obs):
    if obs['career_c'] in career_codes.keys():
        return career_codes[obs['career_c']]
    else:
        return 'Other'

df['career_name'] = df.apply(decode_careers, axis=1)

In [17]:
def demand_satisfaction(obs):
    satis = obs['satis_2']
    if satis <= 2:
        return 'Dissatisfied'
    elif 2 < satis <=4:
        return 'Somewhat Dissatisfied'
    elif 4 < satis <= 6:
        return 'Somewhat Satisfied'
    elif 6 < satis <= 8:
        return 'Satisfied'
    elif 8 < satis:
        return 'Very Satisfied'

df['satisfaction'] = df.apply(demand_satisfaction, axis=1)

In [18]:
def clean_met(obs):
    if obs['met'] == 1:
        return 'Familiar'
    else:
        return 'Strangers'
    
def clean_met_o(obs):
    if obs['met_o'] == 1:
        return 'Familiar'
    else:
        return 'Strangers'

df['met'] = df.apply(clean_met, axis=1)
df['met_o'] = df.apply(clean_met_o, axis=1)

In [19]:
length_codes = {1.0: 'Too short', 2.0: 'Too long', 3.0: 'Just right'}

def decode_length(obs):
    if obs['length'] in list(length_codes.keys()):
        return length_codes[obs['length']]
    
df['length_dc'] = df.apply(decode_length, axis=1)

In [20]:
small_waves = [6,16,18,20]
medium_waves = [1,3,5,7,9,11,12,15]
large_waves = [2,4,6,8,10,13,14,16]

def get_wave_size(obs):
    if obs['wave'] in small_waves:
        return 'small'
    elif obs['wave'] in medium_waves:
        return 'medium'
    elif obs['wave'] in large_waves:
        return 'large'
    
df['wave_size'] = df.apply(get_wave_size, axis=1)

In [21]:
numdat_codes = {1.0: 'Too few', 2.0: 'Too many', 3.0: 'Just right'}

def decode_numdat(obs):
    if obs['numdat_2'] in list(length_codes.keys()):
        return numdat_codes[obs['numdat_2']]
    
df['numdat_dc'] = df.apply(decode_numdat, axis=1)

In [22]:
def get_age_group(obs):
    if obs['age'] < 24:
        return '23 and under'
    elif 24 <= obs['age'] <= 28:
        return '24 to 28'
    elif 28 < obs['age']:
        return '29 and over'
    
df['age_group'] = df.apply(get_age_group, axis=1)

In [23]:
def get_contacted(obs):
    if obs['you_call'] >= 1:
        return True
    elif obs['them_cal'] >= 1:
        return True
    else:
        return False
    
df['contacted'] = df.apply(get_contacted, axis=1)

In [24]:
pref_alloc = ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']
waves_to_scale = [6, 7, 8, 9]
scaled_atbs = {}

for iid in df['iid'].unique():
    scaled_atbs[iid] = {}

def scale_prefs(obs):
    if obs['wave'] in waves_to_scale:
        sum_pref = obs[pref_alloc].sum()
        for atb in pref_alloc:
            try:
                scaled_atbs[obs['iid']][atb] = obs[atb] / sum_pref * 100
            except:
                scaled_atbs[obs['iid']][atb] = np.nan

iids_to_scale = df.query('wave in @waves_to_scale')['iid']

df.apply(scale_prefs, axis=1)

for iid in iids_to_scale:
    print(iid, scaled_atbs[iid])

df.replace(scaled_atbs)

132 {'attr1_1': 16.666666666666664, 'sinc1_1': 16.666666666666664, 'intel1_1': 16.666666666666664, 'fun1_1': 16.666666666666664, 'amb1_1': 16.666666666666664, 'shar1_1': 16.666666666666664}
132 {'attr1_1': 16.666666666666664, 'sinc1_1': 16.666666666666664, 'intel1_1': 16.666666666666664, 'fun1_1': 16.666666666666664, 'amb1_1': 16.666666666666664, 'shar1_1': 16.666666666666664}
132 {'attr1_1': 16.666666666666664, 'sinc1_1': 16.666666666666664, 'intel1_1': 16.666666666666664, 'fun1_1': 16.666666666666664, 'amb1_1': 16.666666666666664, 'shar1_1': 16.666666666666664}
132 {'attr1_1': 16.666666666666664, 'sinc1_1': 16.666666666666664, 'intel1_1': 16.666666666666664, 'fun1_1': 16.666666666666664, 'amb1_1': 16.666666666666664, 'shar1_1': 16.666666666666664}
132 {'attr1_1': 16.666666666666664, 'sinc1_1': 16.666666666666664, 'intel1_1': 16.666666666666664, 'fun1_1': 16.666666666666664, 'amb1_1': 16.666666666666664, 'shar1_1': 16.666666666666664}
133 {'attr1_1': 12.770000000000001, 'sinc1_1': 19.

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,mutual_rate,gender_name,goal_dc,career_name,satisfaction,length_dc,wave_size,numdat_dc,age_group,contacted
0,1,1.0,0,1,1,1,10,7,NaN,4,...,0.500000,Female,To meet new people,Other,Somewhat Satisfied,Too long,medium,Too few,23 and under,True
1,1,1.0,0,1,1,1,10,7,NaN,3,...,0.500000,Female,To meet new people,Other,Somewhat Satisfied,Too long,medium,Too few,23 and under,True
2,1,1.0,0,1,1,1,10,7,NaN,10,...,0.500000,Female,To meet new people,Other,Somewhat Satisfied,Too long,medium,Too few,23 and under,True
3,1,1.0,0,1,1,1,10,7,NaN,5,...,0.500000,Female,To meet new people,Other,Somewhat Satisfied,Too long,medium,Too few,23 and under,True
4,1,1.0,0,1,1,1,10,7,NaN,7,...,0.500000,Female,To meet new people,Other,Somewhat Satisfied,Too long,medium,Too few,23 and under,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8373,552,22.0,1,44,2,21,22,14,10.0,5,...,0.857143,Male,Seemed like a fun night out,Other,Somewhat Satisfied,Too short,None,Too many,24 to 28,True
8374,552,22.0,1,44,2,21,22,13,10.0,4,...,0.857143,Male,Seemed like a fun night out,Other,Somewhat Satisfied,Too short,None,Too many,24 to 28,True
8375,552,22.0,1,44,2,21,22,19,10.0,10,...,0.857143,Male,Seemed like a fun night out,Other,Somewhat Satisfied,Too short,None,Too many,24 to 28,True
8376,552,22.0,1,44,2,21,22,3,10.0,16,...,0.857143,Male,Seemed like a fun night out,Other,Somewhat Satisfied,Too short,None,Too many,24 to 28,True


In [25]:
df_uniq_id = df.drop_duplicates('iid')
df_uniq_id

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,mutual_rate,gender_name,goal_dc,career_name,satisfaction,length_dc,wave_size,numdat_dc,age_group,contacted
0,1,1.0,0,1,1,1,10,7,NaN,4,...,0.500000,Female,To meet new people,Other,Somewhat Satisfied,Too long,medium,Too few,23 and under,True
10,2,2.0,0,3,1,1,10,3,NaN,10,...,0.500000,Female,Seemed like a fun night out,Other,Somewhat Satisfied,Too long,medium,None,24 to 28,False
20,3,3.0,0,5,1,1,10,9,NaN,6,...,NaN,Female,Other,Other,None,None,medium,None,24 to 28,False
30,4,4.0,0,7,1,1,10,6,NaN,3,...,0.666667,Female,Seemed like a fun night out,Lawyer,Somewhat Dissatisfied,Just right,medium,Too many,23 and under,False
40,5,5.0,0,9,1,1,10,4,NaN,1,...,0.333333,Female,To meet new people,Lawyer,Satisfied,Too long,medium,Too many,23 and under,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8268,548,18.0,1,36,2,21,22,21,9.0,13,...,0.555556,Male,Seemed like a fun night out,Banking/Consulting/Finance/Marketing/Business/...,Somewhat Dissatisfied,Just right,None,Too many,29 and over,True
8290,549,19.0,1,38,2,21,22,21,7.0,15,...,0.555556,Male,Seemed like a fun night out,Banking/Consulting/Finance/Marketing/Business/...,Somewhat Satisfied,Too short,None,Too many,24 to 28,True
8312,550,20.0,1,40,2,21,22,21,20.0,2,...,0.571429,Male,To meet new people,Banking/Consulting/Finance/Marketing/Business/...,Satisfied,Just right,None,Just right,29 and over,False
8334,551,21.0,1,42,2,21,22,21,21.0,1,...,0.250000,Male,Seemed like a fun night out,Banking/Consulting/Finance/Marketing/Business/...,Somewhat Dissatisfied,Too short,None,Too many,24 to 28,False


In [26]:
career_counts = df_uniq_id[['career_name']].value_counts().reset_index()
career_counts.columns = ['career_name', 'count']

#career_counts

In [27]:
career_success = df_uniq_id.groupby('career_name')['match_receive_rate'].mean().sort_values(ascending=False).reset_index()
career_success.columns = ['career_name', 'match_receive_rate']
#career_success[['career_name', 'match_receive_rate']]

In [28]:
age_count = df_uniq_id.groupby(['gender', 'age'])['match_receive_rate'].count()
#age_count

In [29]:
age_success = df_uniq_id.groupby('age')['match_receive_rate'].mean()
#age_success

In [30]:
scorecard_ratings = ['attr', 'sinc', 'intel', 'fun', 'amb']
others_ratings = ['attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o']
self_rate_before = ['attr3_1', 'sinc3_1', 'intel3_1', 'fun3_1', 'amb3_1']
self_rate_day_after = ['attr3_2', 'sinc3_2', 'intel3_2', 'fun3_2', 'amb3_2']
self_rate_weeks_after = ['attr3_3', 'sinc3_3', 'intel3_3', 'fun3_3', 'amb3_3']
self_rate_before = ['attr3_1', 'sinc3_1', 'intel3_1', 'fun3_1', 'amb3_1']
others_ratings_avg = df.groupby('pid')[['attr', 'sinc', 'intel', 'fun', 'amb']].mean()


In [31]:
self_ratings = df[['iid','attr3_2', 'sinc3_2', 'intel3_2', 'fun3_2', 'amb3_2']]
self_ratings = self_ratings.drop_duplicates(subset='iid')

all_ratings = self_ratings.merge(others_ratings_avg, left_on='iid', right_on='pid')

all_ratings['diff_attr'] = all_ratings['attr'] - all_ratings['attr3_2']
all_ratings['diff_sinc'] = all_ratings['sinc'] - all_ratings['sinc3_2']
all_ratings['diff_intel'] = all_ratings['intel'] - all_ratings['intel3_2']
all_ratings['diff_fun'] = all_ratings['fun'] - all_ratings['fun3_2']
all_ratings['diff_amb'] = all_ratings['amb'] - all_ratings['amb3_2']

all_ratings.loc[:,'diff_attr':].describe()

,diff_attr,diff_sinc,diff_intel,diff_fun,diff_amb
count,485.000000,485.000000,485.000000,485.000000,485.000000
mean,-0.901542,-0.716509,-0.824920,-1.137294,-0.643552
std,1.532698,1.624215,1.310515,1.518988,1.816832
min,-5.733333,-4.437500,-5.125000,-5.888889,-4.600000
25%,-1.777778,-1.904762,-1.666667,-2.166667,-2.000000
50%,-0.888889,-0.875000,-0.842105,-1.166667,-0.800000
75%,0.000000,0.266667,-0.055556,-0.222222,0.400000
max,4.166667,5.666667,4.300000,5.600000,5.000000


## Visualizations

In [32]:
# histogram of frequency of input column
inputs = ['age','career_name','goal']

for input in inputs:
    fig = px.histogram(df_uniq_id, x=input, color='gender', barmode='overlay')
    fig.show()
    #pyo.plot(fig, filename=input + '_histogram.html')

In [33]:
# pie chart of attendee responses per input
inputs = ['satisfaction', 'length', 'numdat_2']

for input in inputs:
    breakdown_by_input = df_uniq_id.groupby(input)['iid'].count().reset_index()
    fig = px.pie(breakdown_by_input, values='iid', names=input)
    fig.show()
    #pyo.plot(fig, filename=input + '_pie.html')

In [34]:
# bar chart of match receive rate by input
inputs = ['age', 'career_name', 'met_o', 'goal']

for input in inputs:
    match_rate_by_input = df_uniq_id.groupby(['gender', input])['match_receive_rate'].mean().reset_index()

    fig = px.bar(match_rate_by_input, x=input, y='match_receive_rate', color='gender', color_discrete_sequence=['#FF5733', '#33FF7A'], barmode='overlay')
    fig.show()
    #pyo.plot(fig, filename=input + '_match_bar.html')

In [35]:
rating1 = self_rate_day_after
rating2 = self_rate_day_after
rating_diff = {}

for atb in range(0,len(rating1)):
    temp = re.compile("([a-zA-Z]+)([0-9]+)")
    atb_clean = temp.match(rating1[atb]).groups()[0]
    rating_diff[atb_clean + '_diff'] = df[rating2[atb]] - df[rating1[atb]] 

def get_rating_diff(obs):
    for rating in list(rating_diff.keys()):
        print(rating_diff[rating][obs])

#for i in range(0, len(rating_diff)):
#    df[list(rating_diff.keys())[i]] = df.apply(get_rating_diff, axis=1)

'''df.apply(get_rating_diff, axis=1)

df_uniq_id_temp = df.groupby('iid')[list(rating_diff.keys())].mean()

df_uniq_id_temp'''

"df.apply(get_rating_diff, axis=1)\n\ndf_uniq_id_temp = df.groupby('iid')[list(rating_diff.keys())].mean()\n\ndf_uniq_id_temp"

In [36]:
corr_cols = ['shar_o']
for i in range(0,len(others_ratings)):
    corr_cols.append(others_ratings[i])
target = 'dec_o2'
corr1 = df[corr_cols].corrwith(df[target])

corr_cols2 = ['shar']
for i in range(0,len(scorecard_ratings)):
    corr_cols2.append(scorecard_ratings[i])
target2 = 'dec'
corr2 = df[corr_cols2].corrwith(df[target2])

print("Correlations between others' ratings of attendee's attributes and whether or not the other says 'match'\n", corr1)
print("Correlations between attendee's ratings of others' attributes and whether or not they say 'match'\n", corr2)


Correlations between others' ratings of attendee's attributes and whether or not the other says 'match'
 shar_o     0.400501
attr_o     0.486885
sinc_o     0.209811
intel_o    0.216704
fun_o      0.414276
amb_o      0.183216
dtype: float64
Correlations between attendee's ratings of others' attributes and whether or not they say 'match'
 shar     0.400501
attr     0.486881
sinc     0.209811
intel    0.216704
fun      0.414387
amb      0.183216
dtype: float64


In [37]:
# attendees per career
df_uniq_id['career_name'].value_counts()

Academic/Research                                                       152
Banking/Consulting/Finance/Marketing/Business/CEO/Entrepreneur/Admin    144
Lawyer                                                                   47
Creative Arts/Entertainment                                              46
International/Humanitarian Affairs                                       35
Doctor/Medicine                                                          26
Undecided                                                                25
Other                                                                    17
Psychologist                                                             17
Engineer                                                                 14
Social Work                                                              14
Politics                                                                  5
Real Estate                                                               3
Speech Patho

In [38]:
# attendees by age
df_uniq_id['age'].value_counts()

27.0    68
24.0    56
23.0    56
25.0    55
26.0    55
28.0    47
22.0    44
29.0    40
30.0    36
21.0    22
32.0    13
33.0    12
34.0    11
31.0     7
20.0     5
35.0     4
36.0     4
19.0     2
39.0     1
18.0     1
37.0     1
42.0     1
38.0     1
55.0     1
Name: age, dtype: int64

## Creating and Exporting Data for Website Display

In [39]:
orientation = 'index'

### Organizers

#### How do people feel about 4-minutes as a length for each date?

In [52]:
# get value counts for each value in from df_uniq_id
date_length_survey = df_uniq_id['length_dc'].value_counts()
date_length_survey

date_length_survey.to_json('./date_length_survey.json', orient=orientation)

date_length_survey

Too short     269
Just right    202
Too long       14
Name: length_dc, dtype: int64

In [41]:
# show as a bar graph
fig = px.bar(date_length_survey, orientation='h')
#fig.show()

#### How many dates should each person go on?


In [42]:
# count values in numdat_2 and group by wave size
num_dates_survey_small_wave = df_uniq_id[df_uniq_id['wave_size'] == 'small']['numdat_dc'].value_counts()
num_dates_survey_small_wave.to_json('./num_dates_survey_small_wave.json', orient=orientation)

num_dates_survey_medium_wave = df_uniq_id[df_uniq_id['wave_size'] == 'medium']['numdat_dc'].value_counts()
num_dates_survey_medium_wave.to_json('./num_dates_survey_medium_wave.json', orient=orientation)

num_dates_survey_large_wave = df_uniq_id[df_uniq_id['wave_size'] == 'large']['numdat_dc'].value_counts()
num_dates_survey_large_wave.to_json('./num_dates_survey_large_wave.json', orient=orientation)

#### What professions get the most matches from their dates?


In [43]:
# get a sorted list of mean match_receive_rate by career
career_match_success = pd.Series(df_uniq_id.groupby('career_name')['match_receive_rate'].mean().sort_values(ascending=False))
career_match_success.to_json('./career_match_success.json', orient=orientation)

#### Are participants more likely to receive a match if they've met before?


In [44]:
# get mean 'match_receive_rate' by response to 'met_o'
met_before_gender_match_success = df_uniq_id.groupby(['gender_name', 'met_o'])['match_receive_rate'].mean().sort_values(ascending=False)
met_before_gender_match_success.to_json('./met_before_gender_match_success.json', orient=orientation)

met_before_match_success = df_uniq_id.groupby(['met_o'])['match_receive_rate'].mean().sort_values(ascending=False)
met_before_match_success.to_json('./met_before_match_success.json', orient=orientation)

#### What ages get the most matches?


In [45]:
# get a sorted list of mean match_receive_rate by age
age_gender_match_success = df_uniq_id.groupby(['gender_name', 'age'])['match_receive_rate'].mean().sort_values(ascending=False)
age_gender_match_success.to_json('./age_gender_match_success.json', orient=orientation)

age_match_success = df_uniq_id.groupby(['age'])['match_receive_rate'].mean().sort_values(ascending=False)
age_match_success.to_json('./age_match_success.json', orient=orientation)

age_group_gender_match_success = df_uniq_id.groupby(['gender_name', 'age_group'])['match_receive_rate'].mean().sort_values(ascending=False)
age_group_gender_match_success.to_json('./age_group_gender_match_success.json', orient=orientation)

age_group_match_success = df_uniq_id.groupby(['age_group'])['match_receive_rate'].mean().sort_values(ascending=False)
age_group_match_success.to_json('./age_group_match_success.json', orient=orientation)

#### How satisfied were participants with the people they met?


In [46]:
# count values in satisfaction
satisfaction_survey = df_uniq_id['satisfaction'].value_counts()
satisfaction_survey.to_json('./satisfaction_survey.json', orient=orientation)


### Participants


#### What hobbies are most popular among participants?


In [47]:
# get mean hobby ratings and sort
hobby_ratings = df_uniq_id.loc[:,'sports':'yoga'].mean().sort_values(ascending=False)
hobby_ratings.to_json('./hobby_ratings.json', orient=orientation)


#### What is the age of participants?


In [48]:
age_counts = df_uniq_id['age'].value_counts()
age_counts.to_json('./age_counts.json', orient=orientation)


age_group_counts = df_uniq_id['age_group'].value_counts()
age_group_counts.to_json('./age_group_counts.json', orient=orientation)


#### What are participants goals?


In [49]:
# count values in goal_dc
goals_survey = df_uniq_id['goal_dc'].value_counts()
goals_survey.to_json('./goals_survey.json', orient=orientation)

#### How likely are you to match? 
funnel


In [50]:
# get mean of matches logged, mutual matches, a "yes or no" to calls either way, and date
match_to_date_funnel = pd.Series({'Likes': df_uniq_id['matches_given'].mean(),
                                     'Matches': df_uniq_id['mutual_count'].mean(),
                                     'Contact made': df_uniq_id['contacted'].mean(),
                                     'Had a date': df_uniq_id['date_3'].fillna(0).mean()})

match_to_date_funnel.to_json('./match_to_date_funnel.json', orient=orientation)

#### Does self-perception match dates' ratings?


#### How important are these traits?



In [51]:
# get mean of each attributes preference
trait_importance = df[pref_alloc].mean()
trait_importance = trait_importance.set_axis(['Attractiveness', 'Sincerity', 'Intelligence', 'Fun', 'Ambition', 'Shared Interests'])
trait_importance.to_json('./trait_importance.json', orient=orientation)